In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [3]:
class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 3x3 square convolution 
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 3)
        self.conv2 = nn.Conv2d(6, 16, 3)
        # an offline operation: y = Wx+b
        self.fc1 = nn.Linear(16 * 6 * 6, 120) # 6x6 image dimension
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
        
    def num_flat_features(self, x):
        size = x.size()[1:] # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

In [9]:
net = Net()
net

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

The learnable parameters of a model are returned by net.parameters

In [18]:
params = list(net.parameters())
print(len(params))
print(params[0].size()) # conv1's .weight

10
torch.Size([6, 1, 3, 3])


In [20]:
# params[0]

In [22]:
# let try a random 32x32 input
input = torch.randn(1,1,32,32)
input

tensor([[[[-0.8459,  0.4814, -0.6119,  ...,  0.2219,  0.2765, -0.8103],
          [ 0.5658, -0.6574,  1.5637,  ...,  0.9855, -1.4756, -0.0482],
          [-0.9614, -0.9819,  0.2421,  ...,  1.4255, -1.2429,  1.4027],
          ...,
          [-0.2594,  0.3416, -0.4773,  ...,  0.2531,  0.5687, -0.8767],
          [-0.7363, -2.2768,  1.1005,  ..., -0.1538,  0.8494,  0.0205],
          [-2.2814,  2.5322,  1.0436,  ..., -1.2605,  0.7205,  0.4679]]]])

In [23]:
out = net(input)
print(out)

tensor([[-0.0346, -0.0070, -0.0043,  0.0475,  0.1297, -0.0027,  0.1043,  0.0855,
         -0.0746,  0.0221]], grad_fn=<AddmmBackward>)


In [25]:
# Zero the gradient buffers of all parameters and backprops with random gradient
net.zero_grad()
out.backward(torch.randn(1, 10))

#### Loss function

MSE

In [38]:
output = net(input)
target = torch.randn(10) # dummy target, for example


In [39]:
target

tensor([-1.1781, -1.3901,  0.5107, -0.2258, -1.8175, -0.9540,  0.6398,  0.8929,
         0.1114,  0.7184])

In [40]:
target = target.view(1, -1) # make it the same shape as output

In [41]:
target

tensor([[-1.1781, -1.3901,  0.5107, -0.2258, -1.8175, -0.9540,  0.6398,  0.8929,
          0.1114,  0.7184]])

In [42]:
criterion = nn.MSELoss()

In [43]:
loss = criterion(output, target)
print(loss)

tensor(0.9715, grad_fn=<MseLossBackward>)


In [51]:
print(loss.grad_fn) # MSELoss
print(loss.grad_fn.next_functions[0][0]) # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0]) # Relu

#### Backprop

In [52]:
net.zero_grad() # zeros the gradient buffers of all parameters

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward
tensor([2.8253e-02, 4.0200e-03, 1.0684e-02, 2.2978e-03, 5.0383e-03, 9.4132e-05])


#### Update the weights
The simplest update rule used in practice is stochastic gradient descent (SGD)

```weight = weight - learning_rate * gradeient```

In [ ]:
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)

However, as you use neural networks, you want to use various different update rules such as SGD, Nesterov-SGD, Adam, RMSProp, etc. 

In [54]:
import torch.optim as optim

In [55]:
# create optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01)

In [56]:
# in training loop
optimizer.zero_grad() # Zero the gradient buffers
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step() # Does the update